<a href="https://colab.research.google.com/github/hongseoi/colab/blob/main/crime_and_buildingDate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####아파트 가격

####근방 범죄율
- 자치구별범죄현황/자치구별주민등록인구

####건물연식넣기
- 현재 년도(2022)에서 서울시아파트(기준)의 단지 승인일의 연도 빼서 계산


In [ ]:
import pandas as pd
import numpy as np
import json
import urllib.request


### 기준 데이터

In [ ]:
base_df = pd.read_csv("서울시아파트(기준).csv", encoding='cp949')

In [ ]:
base_df.head()

,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,1,A15679103,우리유앤미,아파트,서울특별시 동작구 서달로 83,서울,동작구,흑석동,우리유앤미아파트,서달로,...,위탁,1773.56,223.0,의무,2018-04-10 15:59:42.0,Y,N,126.959639,37.500668,2013-03-07 09:46:59.0
1,2,A13876112,송파파인타운13단지,아파트,서울특별시 송파구 송파대로8길 10,서울,송파구,장지동,857,송파대로8길,...,위탁,0.00,225.0,의무,2013-06-17 19:03:30.0,Y,N,127.129179,37.476897,2013-03-07 09:46:59.0
2,3,A13873701,오금현대백조(임대),아파트,서울특별시 송파구 양재대로72길 20,서울,송파구,오금동,20-2,양재대로72길,...,위탁,6892.79,60.0,임대,2013-11-21 16:43:07.0,Y,N,127.128775,37.508906,2013-03-07 09:46:59.0
3,4,A15275101,개봉건영,아파트,서울특별시 구로구 고척로21나길 85-6,서울,구로구,개봉동,47-1,고척로21나길,...,위탁,0.00,0.0,의무,2013-06-23 11:30:23.0,Y,N,126.840675,37.501162,2013-03-07 09:46:59.0
4,6,A13991016,월계동원베네스트,아파트,서울특별시 노원구 월계로53길 21,서울,노원구,월계동,서울시 노원구 월계2동 940번지,월계로53길,...,위탁,0.00,206.0,의무,2015-06-19 09:59:49.0,Y,N,127.058220,37.631732,2013-03-07 09:46:59.0


In [ ]:
#필요한 데이터만 추출
new_df = base_df[["k-아파트코드","주소(시군구)","단지승인일","좌표X","좌표Y"]]
new_df

,k-아파트코드,주소(시군구),단지승인일,좌표X,좌표Y
0,A15679103,동작구,2018-04-10 15:59:42.0,126.959639,37.500668
1,A13876112,송파구,2013-06-17 19:03:30.0,127.129179,37.476897
2,A13873701,송파구,2013-11-21 16:43:07.0,127.128775,37.508906
3,A15275101,구로구,2013-06-23 11:30:23.0,126.840675,37.501162
4,A13991016,노원구,2015-06-19 09:59:49.0,127.058220,37.631732
...,...,...,...,...,...
2617,A10023859,동작구,2022-04-26 09:48:52.0,126.927512,37.513950
2618,A10023858,용산구,2022-06-07 15:39:48.0,126.962886,37.548159
2619,A10023855,강동구,2022-05-02 11:30:58.0,NaN,NaN
2620,A10023829,강북구,2022-05-09 12:55:00.0,127.033011,37.620171


In [ ]:
#범죄율
crime = pd.read_csv("자치구별범죄현황(2021).csv", encoding='cp949')
people = pd.read_csv("자치구별주민등록인구(2021).csv", encoding='cp949')
crime.head()

,자치구,합계,Unnamed: 2,살인,Unnamed: 4,강도,Unnamed: 6,강간강제추행,Unnamed: 8,절도,Unnamed: 10,폭력,Unnamed: 12
0,NaN,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거,발생,검거
1,합계,92679,66548,141,126,138,132,5763,5458,38293,19705,48344,41127
2,종로구,3102,3343,9,7,7,9,192,855,1183,944,1711,1528
3,중구,3411,2323,2,2,5,6,186,128,1642,863,1576,1324
4,용산구,2969,2056,5,3,6,6,230,186,1111,534,1617,1327


In [ ]:
crime = crime[['자치구','합계']]
crime['범죄수']=crime['합계']
crime = crime.drop('합계',axis=1)
crime = crime.drop([0,1], axis=0)
crime.head()

,자치구,범죄수
2,종로구,3102
3,중구,3411
4,용산구,2969
5,성동구,2362
6,광진구,3601


In [ ]:
people.head()

,자치구,세대,합계,Unnamed: 3,Unnamed: 4,한국인,Unnamed: 6,Unnamed: 7,외국인,Unnamed: 9,Unnamed: 10
0,NaN,NaN,합계,남자,여자,소계,남자,여자,소계,남자,여자
1,서울시,4460604.0,9728414,4711692,5016722,9496887,4607440,4889447,231527,104252,127275
2,종로구,73822.0,153513,73802,79711,143850,69638,74212,9663,4164,5499
3,중구,63880.0,131678,63916,67762,122054,59368,62686,9624,4548,5076
4,용산구,111508.0,236566,114481,122085,222083,106666,115417,14483,7815,6668


In [ ]:
people = people[['자치구','합계']]
people = people.drop([0,1],axis=0)
people.head()

,자치구,합계
2,종로구,153513
3,중구,131678
4,용산구,236566
5,성동구,291051
6,광진구,352903


In [ ]:
crime_rate = pd.merge(crime, people)
crime_rate.head()

,자치구,범죄수,합계
0,종로구,3102,153513
1,중구,3411,131678
2,용산구,2969,236566
3,성동구,2362,291051
4,광진구,3601,352903


In [ ]:
crime_rate["범죄수"] = pd.to_numeric(crime_rate['범죄수'])
crime_rate["합계"] = pd.to_numeric(crime_rate['합계'])

crime_rate["범죄율"] = (crime_rate["범죄수"]/crime_rate["합계"])*100
crime_rate.head()

,자치구,범죄수,합계,범죄율
0,종로구,3102,153513,2.020676
1,중구,3411,131678,2.590410
2,용산구,2969,236566,1.255041
3,성동구,2362,291051,0.811542
4,광진구,3601,352903,1.020394


In [ ]:
#구한 범죄율을 아파트 위치별로 추가하기
new_df['자치구']=new_df['주소(시군구)']
new_df = pd.merge(crime_rate, new_df)
new_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,자치구,범죄수,합계,범죄율,k-아파트코드,주소(시군구),단지승인일,좌표X,좌표Y
0,종로구,3102,153513,2.020676,A11007001,종로구,2013-06-24 11:36:59.0,126.972015,37.573645
1,종로구,3102,153513,2.020676,A11084601,종로구,2013-06-14 16:42:55.0,126.975692,37.609133
2,종로구,3102,153513,2.020676,A11001203,종로구,2013-06-14 16:43:00.0,126.978333,37.610789
3,종로구,3102,153513,2.020676,A11087601,종로구,2013-06-23 15:33:32.0,127.016189,37.573855
4,종로구,3102,153513,2.020676,A11055002,종로구,2017-09-26 11:28:05.0,127.021139,37.575962


### 건물 연식

In [ ]:
new_df

,자치구,범죄수,합계,범죄율,k-아파트코드,주소(시군구),단지승인일,좌표X,좌표Y
0,종로구,3102,153513,2.020676,A11007001,종로구,2013-06-24 11:36:59.0,126.972015,37.573645
1,종로구,3102,153513,2.020676,A11084601,종로구,2013-06-14 16:42:55.0,126.975692,37.609133
2,종로구,3102,153513,2.020676,A11001203,종로구,2013-06-14 16:43:00.0,126.978333,37.610789
3,종로구,3102,153513,2.020676,A11087601,종로구,2013-06-23 15:33:32.0,127.016189,37.573855
4,종로구,3102,153513,2.020676,A11055002,종로구,2017-09-26 11:28:05.0,127.021139,37.575962
...,...,...,...,...,...,...,...,...,...
2617,강동구,3788,465266,0.814158,A10024420,강동구,2021-03-09 17:13:37.0,127.144817,37.554724
2618,강동구,3788,465266,0.814158,A10024421,강동구,2021-04-13 17:44:41.0,127.173308,37.570719
2619,강동구,3788,465266,0.814158,A10024412,강동구,2021-03-09 14:59:43.0,127.144817,37.554724
2620,강동구,3788,465266,0.814158,A10023979,강동구,2022-03-08 16:39:36.0,127.128701,37.538079


In [ ]:
new_df.isnull().sum()

자치구         0
범죄수         0
합계          0
범죄율         0
k-아파트코드     0
주소(시군구)     0
단지승인일       0
좌표X        11
좌표Y        11
dtype: int64

In [ ]:
new_df['단지승인연도'] = new_df['단지승인일'].str[:4]
new_df.head()

,자치구,범죄수,합계,범죄율,k-아파트코드,주소(시군구),단지승인일,좌표X,좌표Y,단지승인연도
0,종로구,3102,153513,2.020676,A11007001,종로구,2013-06-24 11:36:59.0,126.972015,37.573645,2013
1,종로구,3102,153513,2.020676,A11084601,종로구,2013-06-14 16:42:55.0,126.975692,37.609133,2013
2,종로구,3102,153513,2.020676,A11001203,종로구,2013-06-14 16:43:00.0,126.978333,37.610789,2013
3,종로구,3102,153513,2.020676,A11087601,종로구,2013-06-23 15:33:32.0,127.016189,37.573855,2013
4,종로구,3102,153513,2.020676,A11055002,종로구,2017-09-26 11:28:05.0,127.021139,37.575962,2017


In [ ]:
new_df["단지승인연도"] = pd.to_numeric(new_df['단지승인연도'])


In [ ]:
new_df['건물연식'] = abs(new_df['단지승인연도'].sub(2022))
new_df

,자치구,범죄수,합계,범죄율,k-아파트코드,주소(시군구),단지승인일,좌표X,좌표Y,단지승인연도,건물연식
0,종로구,3102,153513,2.020676,A11007001,종로구,2013-06-24 11:36:59.0,126.972015,37.573645,2013,9
1,종로구,3102,153513,2.020676,A11084601,종로구,2013-06-14 16:42:55.0,126.975692,37.609133,2013,9
2,종로구,3102,153513,2.020676,A11001203,종로구,2013-06-14 16:43:00.0,126.978333,37.610789,2013,9
3,종로구,3102,153513,2.020676,A11087601,종로구,2013-06-23 15:33:32.0,127.016189,37.573855,2013,9
4,종로구,3102,153513,2.020676,A11055002,종로구,2017-09-26 11:28:05.0,127.021139,37.575962,2017,5
...,...,...,...,...,...,...,...,...,...,...,...
2617,강동구,3788,465266,0.814158,A10024420,강동구,2021-03-09 17:13:37.0,127.144817,37.554724,2021,1
2618,강동구,3788,465266,0.814158,A10024421,강동구,2021-04-13 17:44:41.0,127.173308,37.570719,2021,1
2619,강동구,3788,465266,0.814158,A10024412,강동구,2021-03-09 14:59:43.0,127.144817,37.554724,2021,1
2620,강동구,3788,465266,0.814158,A10023979,강동구,2022-03-08 16:39:36.0,127.128701,37.538079,2022,0


In [ ]:
new_df.to_csv('C:\\Users\\HONG SEO I\\crime_and_buildDate.csv', index = None)